<h1> Group 20 - DMML Coursework 2 <h1>
    
<hr>

# Data
Setup full data set:

In [63]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
# Common imports
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

#read the attribute csvs file into respective dataframes
X = pd.read_csv ('x_train_gr_smpl.csv')


# read the class attribute file into Y datafram
Y = pd.read_csv('y_train_smpl.csv')
Y.columns = ['Class']

#join the two such that Y is the last column
XYraw = pd.concat([X,Y], axis=1)
#check the concatination
print(XYraw)

          0      1      2      3      4      5      6      7      8      9  \
0      78.0   77.0   76.0   82.0   87.0   92.0  104.0  119.0  117.0  120.0   
1      73.0   75.0   79.0   78.0   76.0   75.0   89.0  107.0  133.0  125.0   
2      72.0   75.0   79.0   77.0   81.0   89.0  105.0  109.0   86.0   90.0   
3      67.0   70.0   74.0   80.0   93.0  107.0  110.0   96.0   69.0  100.0   
4      74.0   74.0   73.0   72.0   77.0   87.0  104.0  109.0   84.0   83.0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
9685  247.0  220.0  254.0  213.0  129.0  208.0  254.0  255.0  255.0  255.0   
9686  151.0  118.0  254.0  255.0  255.0  255.0  254.0  254.0  254.0  252.0   
9687  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0   
9688  255.0  253.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0  255.0   
9689  252.0  189.0  238.0  255.0  255.0  245.0  219.0  212.0  140.0   40.0   

      ...  2295  2296   2297  2298  2299  2300   2301   2302   

In [64]:
#randomise 
XYraw_ran = XYraw.sample(frac=1).reset_index(drop=True)
print(XYraw_ran)

          0      1      2      3      4      5      6      7      8      9  \
0      22.0   23.0   25.0   24.0   24.0   26.0   25.0   24.0   23.0   23.0   
1      85.0   83.0   82.0   81.0   80.0   79.0   80.0   82.0   83.0   84.0   
2      17.0   14.0   26.0   29.0   31.0   28.0   25.0   18.0   49.0   73.0   
3      22.0   22.0   22.0   22.0   23.0   26.0   31.0   35.0   38.0   35.0   
4     177.0  177.0  179.0  178.0  177.0  175.0  175.0  177.0  177.0  178.0   
...     ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
9685   23.0   22.0   24.0   22.0   21.0   24.0   22.0   18.0   16.0   16.0   
9686   27.0   25.0   28.0   37.0   36.0   32.0   27.0   21.0   19.0   19.0   
9687   73.0   82.0   91.0  122.0  124.0  120.0  112.0   98.0   90.0   89.0   
9688  156.0  158.0  161.0  161.0  157.0  154.0  155.0  153.0  154.0  155.0   
9689   30.0   32.0   28.0   51.0  176.0  232.0  145.0   78.0   77.0  122.0   

      ...   2295   2296   2297   2298   2299   2300   2301   23

In [65]:
#generate this as a csv to directly feed into analyses on independent notebooks
XYraw_ran.to_csv('For_10_Fold_Cross_Validation.csv', index = False)

Split Data into data and expected classes:

In [66]:
#read in data from csv 
training_Data = pd.read_csv("For_10_Fold_Cross_Validation.csv")

#look at shape
training_Data.shape 

#look at data format
training_Data.head()

,0,1,2,3,4,5,6,7,8,9,...,2295,2296,2297,2298,2299,2300,2301,2302,2303,Class
0,22.0,23.0,25.0,24.0,24.0,26.0,25.0,24.0,23.0,23.0,...,23.0,24.0,23.0,22.0,23.0,24.0,25.0,24.0,23.0,3
1,85.0,83.0,82.0,81.0,80.0,79.0,80.0,82.0,83.0,84.0,...,62.0,58.0,57.0,56.0,54.0,52.0,52.0,52.0,54.0,1
2,17.0,14.0,26.0,29.0,31.0,28.0,25.0,18.0,49.0,73.0,...,16.0,13.0,11.0,10.0,11.0,11.0,12.0,14.0,17.0,2
3,22.0,22.0,22.0,22.0,23.0,26.0,31.0,35.0,38.0,35.0,...,23.0,25.0,26.0,26.0,25.0,26.0,26.0,27.0,27.0,1
4,177.0,177.0,179.0,178.0,177.0,175.0,175.0,177.0,177.0,178.0,...,12.0,13.0,13.0,13.0,14.0,15.0,17.0,16.0,15.0,3


In [67]:
#define the data to be used as all but the class attribute from the data 
X_Train = training_Data.drop('Class', axis = 1)

#define the expexted value to be the class column of the input data 
y_train = training_Data['Class']

Scale data:

In [68]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_Train)

<b> Setup Test Data </b>

Import test data from file and set it as t_test and y_test

In [69]:
testData = pd.read_csv ('Test_Dataset_Linear.csv')

X_Test = testData.drop('Class', axis = 1)
y_test = testData['Class']


X_test = sc.fit_transform(X_Test)

<b>Setup 4000 Split of test data</b>

Take 4000 instances from the original training file and add them to the test file

In [70]:
#Check length of training data
len(training_Data)

9690

In [71]:
#split data into two divisions
data_split4000, data_split5690= training_Data[:4000], training_Data[4000:]

#test new split 
len(data_split4000)

4000

In [72]:
#merge test and 4000 cases from above 
testData_with4000 =pd.concat([testData, data_split4000])

In [73]:
#size of testData before addition
len(testData)

3090

In [74]:
#check items have been added to testData
len(testData_with4000)

7090

In [75]:
#setup training data
X_Train_4000Split = data_split5690.drop('Class', axis = 1)
y_train_4000Split = data_split5690['Class']


#setup test data
X_Test_4000Split = testData_with4000.drop('Class', axis = 1)
y_test_4000Split = testData_with4000['Class']



sc = StandardScaler()
X_train_4000Split = sc.fit_transform(X_Train_4000Split)
X_test_4000Split = sc.fit_transform(X_Test_4000Split)

<b>Setup 9000 Split of test data</b>

Take 9000 instances from the original training file and add them to the test file

In [76]:
#Check length of training data
len(training_Data)

9690

In [77]:
#split data into two divisions
data_split9000, data_split690= training_Data[:9000], training_Data[9000:]

#test new split 
len(data_split9000)

9000

In [78]:
#merge test and 4000 cases from above 
testData_with9000 =pd.concat([testData, data_split9000])

In [79]:
#size of testData before addition 
len(testData)

3090

In [80]:
#check items have been added to testData
len(testData_with9000)

12090

In [81]:
#setup training data
X_Train_9000Split = data_split690.drop('Class', axis = 1)
y_train_9000Split = data_split690['Class']


#setup test data
X_Test_9000Split = testData_with4000.drop('Class', axis = 1)
y_test_9000Split = testData_with4000['Class']

#scale data
X_train_9000Split = sc.fit_transform(X_Train_9000Split)
X_test_9000Split = sc.fit_transform(X_Test_9000Split)

<h2> Neural Networks <h2>
   <hr>

<h3> Linear Classifier <h3>

In [82]:
from sklearn import linear_model
import pandas as pd
import numpy as np

In [83]:
regr = linear_model.LinearRegression()

In [84]:
regr.fit(X_train, y_train)

LinearRegression()

In [85]:
print(regr.coef_)

[-0.22573113  0.48597554 -0.76214387 ... -0.41743961  0.12059639
  0.01184512]


In [86]:
np.mean((regr.predict(X_test) - y_test)**2)

3.449489488059733

In [88]:
# Explained variance score: 1 is perfect prediction
# and 0 means that there is no linear relationship
# between X and y.
regr.score(X_test, y_test)

0.22849361262331425